In [ ]:
def definetolerance(grid_mask, grid_dem):
	# This function defines the tolerance when the automatic mode is used
	
	# Create grids of x and y in a local reference system
	s2=grid_dem.shape
	grid_y, grid_x = np.mgrid[0:s2[0]*cellSize:1*cellSize,0:s2[1]*cellSize:1*cellSize]
	grid_y = np.flipud(grid_y)
	
	if verbose:
		str_message = 'Altitude of cell [0,0] = {}'.format(grid_dem[0,0])
		arcpy.AddMessage(str_message)
	
	# Finds the central point of the polygon
	x_mean = np.mean(grid_x[grid_mask])
	y_mean = np.mean(grid_y[grid_mask])
	
	if verbose:
		str_message = 'x_mean = {}, y_mean = {}'.format(x_mean,y_mean)
		arcpy.AddMessage(str_message)
		str_message = 'x: [{}-{}], y: [{}-{}]'.format(np.min(grid_x[grid_mask]),np.max(grid_x[grid_mask]),np.min(grid_y[grid_mask]),np.max(grid_y[grid_mask]))
		arcpy.AddMessage(str_message)
	
	# Vectorizes the grids for the points inside the polygon
	x = grid_x[grid_mask].flatten()
	y = grid_y[grid_mask].flatten()
	z = grid_dem[grid_mask].flatten()
	
	x = np.expand_dims(x,axis=1)
	y = np.expand_dims(y,axis=1)
	z = np.expand_dims(z,axis=1)
	
	# Fits a plan on the surface of the landslide
	# Inspired from: https://gist.github.com/RustingSword/e22a11e1d391f2ab1f2c
	XY1 = np.concatenate((x,y,np.ones(x.shape)),axis=1)
	if verbose:
		str_message = 'XY1({},{}) is {} with range {}-{}, z({},{}) is {} with range {}-{}'.format(XY1.shape[0],XY1.shape[1],type(XY1),np.min(XY1),np.max(XY1),z.shape[0],z.shape[1],type(z),np.min(z),np.max(z))
		arcpy.AddMessage(str_message)
	(a, b, c),resid,rank,s = np.linalg.lstsq(XY1, z, rcond=1e-10)
	
	# Calculates the normal vector of the plan
	normal = np.array([a[0], b[0], -1])
	nn = np.linalg.norm(normal)
	normal = normal / nn
	
	# Ensures that the vector points upwards
	if normal[2] < 0:
		normal = normal * -1
		
	if verbose:
		str_message = 'normal vector ({},{},{}), c = {}'.format(normal[0],normal[1],normal[2],c[0])
		arcpy.AddMessage(str_message)
		
		# https://se.mathworks.com/matlabcentral/answers/342134-how-can-i-find-strike-and-dip-of-a-plane-from-equation-of-a-plane
		n_e = np.array([0,0,1])
		dip = np.degrees(np.arccos(np.dot(normal,n_e)))
		dip_dir = np.degrees(np.arctan2(normal[0], normal[1]))
		
		str_message = 'dip dir = {}, dip {}'.format(dip_dir,dip)
		arcpy.AddMessage(str_message)

	if savefigs:
		# plot points and fitted surface
		from mpl_toolkits.mplot3d import Axes3D
		import matplotlib.pyplot as plt
		fig = plt.figure()
		ax = fig.gca(projection='3d')
		z_plane = a[0]*x + b[0]*y + c[0]
		ax.scatter(x, y, z_plane, c='b', s=5)
		ax.scatter(x, y, z, c='r', s=5)
		plt.xlabel('X')
		plt.ylabel('Y')
		ax.set_zlabel('Z')
		ax.axis('equal')
		ax.axis('tight')
		save_name = validatename(figspath,'plane','.png')
		fig.savefig(os.path.join(figspath,save_name))
		plt.close()
	
	# finds the slope and y-intercept of the profile in the xy plan
	slope = normal[1]/normal[0]
	y0 = y_mean - (slope * x_mean)
		
	xmin = 0
	ymin = 0
	xmax = arcpy.env.extent.XMax - arcpy.env.extent.XMin
	ymax = arcpy.env.extent.YMax - arcpy.env.extent.YMin
	
	if verbose:
		str_message = 'xmax: {}, ymax: {}, y0: {}'.format(xmax,ymax,y0)
		arcpy.AddMessage(str_message)
	
	# finds the coordinates (xa,ya) and (xb,yb) formed by the intersection of
	# the profile with the study area
	y_xmin = xmin * slope + y0
	y_xmax = xmax * slope + y0
	x_ymin = (ymin - y0)/slope
	x_ymax = (ymax - y0)/slope
	
	if slope < 0:
		if x_ymax > xmin:
			xa = x_ymax
			ya = ymax
		else:
			xa = xmin
			ya = y_xmin
		if x_ymin < xmax:
			xb = x_ymin
			yb = ymin
		else:
			xb = xmax
			yb = y_xmax
	else:
		if x_ymin > xmin:
			xa = x_ymin
			ya = ymin
		else:
			xa = xmin
			ya = y_xmin
		if x_ymax < xmax:
			xb = x_ymax
			yb = ymax
		else:
			xb = xmax
			yb = y_xmax
	
	if verbose:
		str_message = 'profile: ({},{})-({},{})'.format(xa,ya,xb,yb)
		arcpy.AddMessage(str_message)
	
	#-- Interpolate the altitudes on the profile
	# Vectorizes the grids for all the points
	x = grid_x.flatten()
	y = grid_y.flatten()
	z = grid_dem.flatten()
	
	x = np.expand_dims(x,axis=1)
	y = np.expand_dims(y,axis=1)
	z = np.expand_dims(z,axis=1)
	
	# Generate points evenly spaced along the profile
	x_profile = np.linspace(xa,xb,num=100)
	y_profile = np.linspace(ya,yb,num=100)
	xy_profile = np.column_stack((x_profile,y_profile))
	
	l_profile = np.sqrt((x_profile-xa)**2 + (y_profile-ya)**2)
	
	XY = np.concatenate((x,y),axis=1)
	
	profil_int = griddata(XY, z, xy_profile, method='cubic')
	mask_int = griddata(XY, grid_mask.flatten(), xy_profile, method='nearest')
	
	if savefigs:
		# plot profile
		import matplotlib.pyplot as plt
		fig = plt.figure()
		ax = fig.gca()
		ax.plot(l_profile,profil_int,'b')
		ax.plot(l_profile[mask_int],profil_int[mask_int],'r')
		ax.axis('equal')
		ax.axis('tight')
		save_name = validatename(figspath,'profile','.png')
		fig.savefig(os.path.join(figspath,save_name))
		plt.close()

	# Calculate the tolerance using the altitude range of the profile (dZ) and its length (dL)
	dZ = np.max(profil_int[mask_int]) - np.min(profil_int[mask_int])
	dL = np.max(l_profile[mask_int]) - np.min(l_profile[mask_int])
	tol_max = 4 * (1-np.sqrt(2)) * dZ * (cellSize**2/dL**2)

	if verbose:
		arcpy.AddMessage('dZ={} ({}-{}), dL={}'.format(dZ,profil_int[0],profil_int[-1],dL))
		arcpy.AddMessage('tol max: {}'.format(tol_max))
	
	tol_inter = 2 * (1-np.sqrt(2)) * dZ * (cellSize**2/dL**2)
	tol_min = 0	
	
	return tol_inter, tol_min, tol_max